In [25]:
import silq
silq.initialize('Serwan')

Initializing imports
Initialization complete


In [32]:
import logging
logger = logging.getLogger()
logger.level = logging.DEBUG

In [2]:
from IPython.core.magic import Magics, magics_class, line_magic

In [82]:
p_dummy = ManualParameter('dummy')
Loop(p_dummy[0:10:1]).each(p_dummy).run('test')

DataSet:
   data = load_data('data/2017-05-28/#001_{name}_17-48-11')
   <Type>   | <array_id> | <array.name> | <array.shape>
   Setpoint | dummy_set  | dummy        | (10,)
   Measured | dummy      | dummy        | (10,)
Started at 2017-05-28 17:48:11
Finished at 2017-05-28 17:48:11


DataSet:
   data = load_data('data/2017-05-28/#001_{name}_17-48-11')
   <Type>   | <array_id> | <array.name> | <array.shape>
   Setpoint | dummy_set  | dummy        | (10,)
   Measured | dummy      | dummy        | (10,)

In [91]:
base_path = config.properties.data_folder
base_path
os.path.split(base_path)

('C:\\Users\\Serwan\\Documents', 'data')

In [110]:
from dateutil.parser import parse

@magics_class
class CustomMagics(Magics):
    """Magics related to code management (loading, saving, editing, ...)."""

    def __init__(self, *args, **kwargs):
        self._knowntemps = set()
        super(CustomMagics, self).__init__(*args, **kwargs)

    @line_magic
    def data(self, line):
        line = line.replace('\\', '/')
        
        base_path = config.properties.data_folder
        base_path = base_path.replace('\\', '/')
        base_folder = os.path.split(base_path)[1]
        
        logging.debug(f'Dirs: {os.listdir(base_path)}')
        
        if '/' not in line:
            logging.debug('No date provided, getting latest date folder')
            date_folders = reversed(os.listdir(base_path))
            for date_folder in date_folders:
                logging.debug(f'Date folder: {date_folder}')
                try:
                    parse(date_folder)
                    # date_folder has date format, exit loop
                    break
                except:
                    # date_folder does not have date format, continuing to next
                    continue
            else:
                raise RuntimeError(f'No date folder found in {base_path}')
            # No data arg provided, any data folder will match data_str
            data_str = ''
        elif ':' in line:
            raise NotImplementedError('Full paths not yet implemented')
        else:
            date_folder, data_str = line.split('/')[-2:]
        date_path = os.path.join(base_path, date_folder)
        
        data_folders = reversed(os.listdir(date_path))
        data_folder = next(folder for folder in data_folders if data_str in folder)
        data_path = os.path.join(base_folder, date_folder, data_folder)
        logging.debug(f'Data path: {data_path}')

        

In [111]:
ip = get_ipython()
mm = ip.magics_manager
mm.register(CustomMagics)

In [112]:
%data #425

DEBUG:root:Dirs: ['2016-12-11', '2016-12-23', '2016-12-26', '2016-12-31', '2017-01-01', '2017-01-02']
DEBUG:root:No date provided, getting latest date folder
DEBUG:root:Date folder: 2017-01-02
DEBUG:root:Data path: data\2017-01-02\#425_track_dummy_gaussian_12-33-50
DEBUG:root:<ipykernel.zmqshell.ZMQInteractiveShell object at 0x000002A60F1200B8>
